# Data Mining Project
---

Import βιβλιοθήκες

In [ ]:
from pandas.io.sas.sas_constants import column_data_offset_offset

In [ ]:
import pandas as pd
import numpy as np

### **TODO** Να δείξουμε πως από txt το κάναμε csv.

Converting to dataframe

In [ ]:
df = pd.read_csv('household_power_consumption.csv')

In [ ]:
df.head()

Συμπλήρωση ελλιπών τιμών με ffill()

In [ ]:
df.ffill(inplace=True)

In [ ]:
print(df.isna().sum())

Μετατροπή σε Datetime

In [ ]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

print(df.head())

Επαναδειγματοληψία (Resampling)

In [ ]:
df_daily = df.resample('D').agg({ # D = daily
    'Global_active_power': 'sum',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum'
})

Δημιουργία Daily_total_power

In [ ]:
df_daily.rename(columns={'Global_active_power': 'Daily_total_power'}, inplace=True)

Δημιουργία Peak_hour_power

In [ ]:
hourly_consumption = df['Global_active_power'].resample('h').sum() # h = hourly
peak_hour_series = hourly_consumption.resample('D').max()

Δημιουργία Nighttime_usage

In [ ]:
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
df_night = df[night_mask]
night_usage_series = df_night['Global_active_power'].resample('D').sum()

Προσθήκη χαρακτηριστικών ως columns

In [ ]:
df_daily['Peak_hour_power'] = peak_hour_series
df_daily['Night_usage_power'] = night_usage_series

Κατηγορικά χαρακτηριστικά

In [40]:
df_daily['Day_of_week'] = df_daily.index.dayofweek
df_daily['Is_workday'] = df_daily['Day_of_week'].apply(lambda x: 1 if x < 5 else 0) # Δευτέρα - Παρασκευή
df_daily['Is_weekend'] = df_daily['Day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # ΣΚ
df_daily['Month'] = df_daily.index.month

In [41]:
def get_season(month):
    if month in [12,1,2]: return 1 # Winter
    elif month in [3,4,5]: return 2 # Spring
    elif month in [6,7,8]: return 3 # Summer
    else: return 4 # Fall

df_daily['Season'] = df_daily['Month'].apply(get_season)

Δημιουργία Weekend_usage

In [42]:
df_daily['Weekend_usage'] = df_daily.apply(
    lambda row: row['Daily_total_power'] if row['Is_weekend'] == 1 else 0,
    axis=1
)

In [43]:
df_daily.fillna(0, inplace=True)

In [44]:
print(df_daily.head())

            Daily_total_power  Global_reactive_power     Voltage  \
datetime                                                           
2006-12-16           1209.176               0.088187  236.243763   
2006-12-17           3390.460               0.156949  240.087028   
2006-12-18           2203.826               0.112356  241.231694   
2006-12-19           1666.194               0.104821  241.999313   
2006-12-20           2225.748               0.111804  242.308062   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
datetime                                                                       
2006-12-16         13.082828             0.0           546.0          4926.0   
2006-12-17          9.999028          2033.0          4187.0         13341.0   
2006-12-18          6.421667          1063.0          2621.0         14018.0   
2006-12-19          4.926389           839.0          7602.0          6197.0   
2006-12-20          6.467361             0.